# Key Components of this Implementation

## Data Preprocessing

- **Normalization and Resizing**: Input images are normalized to [0, 1] and resized to 224x224 using a Lanczos filter.
- **Augmentation**: SimCLR uses strong augmentations such as:
  - Random resized cropping
  - Horizontal flipping
  - Color jittering
  - Gaussian blur  
  These augmentations create two distinct views of each image.

## Dataset

- A `SimCLRDataset` class generates paired augmented views `(img_1, img_2)` for the unsupervised contrastive learning phase.

## SimCLR Model

- **Backbone (Encoder)**: ResNet-18 pretrained on ImageNet, modified to accept single-channel images.
- **Projection Head**: A small fully connected network with two layers and ReLU activation, mapping features from the encoder to a lower-dimensional latent space.

## NT-Xent Loss (Contrastive Loss)

- Encourages similar views of the same image to have a higher similarity, while dissimilar pairs (views from different images) have a lower similarity.
- Uses normalized temperature-scaled cross-entropy loss.

## Unsupervised Pretraining

- The model is trained using NT-Xent loss in an unsupervised manner, leveraging the augmented pairs of images without labels.

## Supervised Fine-Tuning

- A classification head is added to the encoder, and both are fine-tuned using labeled training data for the classification task.

## Testing

- The model's encoder and classification head are evaluated on the test dataset for classification accuracy.

---

# Training Details of the SimCLR Model

## Training Parameters

- **Batch Size**: 
  - 512 for SimCLR pretraining 
  - 128 for supervised fine-tuning classification task
- **Learning Rate**:
  - 1e-4 for the SimCLR model during pretraining.
  - 1e-5 for the encoder and 1e-3 for the classification head during fine-tuning.
- **Optimizer**: Adam optimizer.
- **Scheduler**: StepLR with a decay factor of 0.5 every 10 epochs (fine-tuning phase).

## Augmentation Strategy

- Contrastive learning heavily relies on strong augmentations to create diverse views of the same image for robust feature learning.

## Unsupervised Pretraining

- Pretrained for 100 epochs using the NT-Xent loss on the unlabeled training dataset.

## Supervised Fine-Tuning

- Trained for 100 epochs using cross-entropy loss on labeled training data to classify the brain images.

## Evaluation

- Accuracy is calculated on the test dataset after fine-tuning the classification head.


# PreProcessing

In [2]:
# Section 1: Import Libraries
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import matplotlib.pyplot as plt

# Section 2: Load and Preprocess Dataset
train_images_path = "brain_train_image_final.npy"
train_labels_path = "brain_train_label.npy"
test_images_path = "brain_test_image_final.npy"
test_labels_path = "brain_test_label.npy"

# Load the data
final_X_train_modified = np.load(train_images_path)[:, 1, :, :]
final_X_test_modified = np.load(test_images_path)[:, 1, :, :]
train_labels = np.load(train_labels_path)
test_labels = np.load(test_labels_path)

# Normalize and Resize Images using Pillow
def normalize_and_resize(images, target_size=(224, 224)):
    resized_images = []
    for img in images:
        img = Image.fromarray((img * 255).astype(np.uint8))
        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
        resized_images.append(np.array(img_resized) / 255.0)
    return np.array(resized_images)

final_X_train_resized = normalize_and_resize(final_X_train_modified)
final_X_test_resized = normalize_and_resize(final_X_test_modified)


# SimCLR Augmentation

In [4]:
# Section 3: Define SimCLR Augmentation and Dataset
transform_simclr = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(size=224, scale=(0.08, 1.0), ratio=(3/4, 4/3)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.8, contrast=0.8, saturation=0.8, hue=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0))], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class SimCLRDataset(Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img_1 = self.transform(img)
        img_2 = self.transform(img)
        return img_1, img_2

train_dataset = SimCLRDataset(final_X_train_resized, transform_simclr)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)


In [5]:
import os
from PIL import Image

# Section 4: Save Augmented Images
def save_augmented_images(dataset, save_dir="augmented_images", num_images=5):
    """
    Save a set of augmented images from the dataset to a directory.
    """
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    for i in range(num_images):
        img_original = dataset.images[i]  # Original image
        img_aug1, img_aug2 = dataset[i]  # Augmented pair

        # Unnormalize images
        img_aug1 = transforms.ToPILImage()(img_aug1 * 0.5 + 0.5)
        img_aug2 = transforms.ToPILImage()(img_aug2 * 0.5 + 0.5)

        # Save images
        img_aug1.save(os.path.join(save_dir, f"image_{i}_aug1.png"))
        img_aug2.save(os.path.join(save_dir, f"image_{i}_aug2.png"))

        print(f"Saved image_{i}_aug1.png and image_{i}_aug2.png to {save_dir}")

# Save augmented images from the training dataset
save_augmented_images(train_dataset, save_dir="augmented_images", num_images=5)


Saved image_0_aug1.png and image_0_aug2.png to augmented_images
Saved image_1_aug1.png and image_1_aug2.png to augmented_images
Saved image_2_aug1.png and image_2_aug2.png to augmented_images
Saved image_3_aug1.png and image_3_aug2.png to augmented_images
Saved image_4_aug1.png and image_4_aug2.png to augmented_images


# Training on Brain train images with batch size = 512

In [6]:
# Section 4: Define SimCLR Model and NT-Xent Loss
class SimCLR(nn.Module):
    def __init__(self, base_encoder, projection_dim):
        super(SimCLR, self).__init__()
        self.encoder = base_encoder
        self.projector = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, projection_dim)
        )

    def forward(self, x):
        h = self.encoder(x)
        z = self.projector(h)
        return z

class NTXentLoss(nn.Module):
    def __init__(self, batch_size, temperature):
        super(NTXentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss(reduction="sum")

    def forward(self, z_i, z_j):
        N = z_i.size(0) + z_j.size(0)
        z = torch.cat((z_i, z_j), dim=0)
        sim = torch.matmul(z, z.T) / self.temperature
        mask = ~torch.eye(N, dtype=torch.bool, device=z.device)

        positives = torch.cat([
            torch.diag(sim, z_i.size(0)),
            torch.diag(sim, -z_i.size(0))
        ])

        negatives = sim[mask].view(N, -1)
        logits = torch.cat((positives.unsqueeze(1), negatives), dim=1)
        labels = torch.zeros(N, dtype=torch.long, device=z.device)
        loss = self.criterion(logits, labels) / N
        return loss

# Section 5: Initialize and Train SimCLR Model
base_encoder = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
base_encoder.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
base_encoder.fc = nn.Identity()

model = SimCLR(base_encoder, projection_dim=128).to("cuda")
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = NTXentLoss(batch_size=512, temperature=0.5)

for epoch in range(100):
    total_loss = 0
    model.train()
    for img_1, img_2 in train_loader:
        img_1, img_2 = img_1.to("cuda"), img_2.to("cuda")
        z_i = model(img_1)
        z_j = model(img_2)

        loss = criterion(z_i, z_j)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}")

Epoch [1/100], Loss: 8.2772
Epoch [2/100], Loss: 6.8251
Epoch [3/100], Loss: 6.6696
Epoch [4/100], Loss: 6.6111
Epoch [5/100], Loss: 6.5437
Epoch [6/100], Loss: 6.5174
Epoch [7/100], Loss: 6.4632
Epoch [8/100], Loss: 6.4140
Epoch [9/100], Loss: 6.3949
Epoch [10/100], Loss: 6.3732
Epoch [11/100], Loss: 6.2921
Epoch [12/100], Loss: 6.2366
Epoch [13/100], Loss: 6.2183
Epoch [14/100], Loss: 6.1661
Epoch [15/100], Loss: 6.0848
Epoch [16/100], Loss: 5.9737
Epoch [17/100], Loss: 5.9455
Epoch [18/100], Loss: 5.8610
Epoch [19/100], Loss: 5.8191
Epoch [20/100], Loss: 5.7111
Epoch [21/100], Loss: 5.6278
Epoch [22/100], Loss: 5.5041
Epoch [23/100], Loss: 5.3868
Epoch [24/100], Loss: 5.3143
Epoch [25/100], Loss: 5.2859
Epoch [26/100], Loss: 5.1270
Epoch [27/100], Loss: 5.0033
Epoch [28/100], Loss: 4.9615
Epoch [29/100], Loss: 4.8229
Epoch [30/100], Loss: 4.8235
Epoch [31/100], Loss: 4.6408
Epoch [32/100], Loss: 4.5560
Epoch [33/100], Loss: 4.4728
Epoch [34/100], Loss: 4.4754
Epoch [35/100], Loss: 4

In [5]:
# Section 6: Save the Pretrained SimCLR Model
import os

# Define the path to save the model
save_path = "simclr_pretrained_18_512.pth"

# Save the model's state dictionary
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
    'loss': total_loss / len(train_loader)
}, save_path)

print(f"Pretrained SimCLR model saved to {save_path}")


Pretrained SimCLR model saved to simclr_pretrained_18_512.pth


# Testing on Brain test images

In [ ]:
# Section 7: Define Classification Dataset and Head
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = TestDataset(final_X_train_resized, train_labels, transform=train_transform)
test_dataset = TestDataset(final_X_test_resized, test_labels, transform=test_transform)

class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

classification_head = ClassificationHead(input_dim=512, num_classes=len(np.unique(train_labels))).to("cuda")

# Section 8: Fine-tune and Evaluate Classification Model
optimizer_cls = optim.Adam([
    {"params": model.encoder.parameters(), "lr": 1e-5},
    {"params": classification_head.parameters(), "lr": 1e-3},
])
scheduler_cls = StepLR(optimizer_cls, step_size=10, gamma=0.5)
criterion_cls = nn.CrossEntropyLoss()

# Fine-tune
for epoch in range(100):
    model.encoder.train()
    classification_head.train()
    total_loss = 0
    correct = 0
    for img, label in DataLoader(train_dataset, batch_size=128, shuffle=True):
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        loss = criterion_cls(logits, label)

        optimizer_cls.zero_grad()
        loss.backward()
        optimizer_cls.step()

        total_loss += loss.item()
        correct += (logits.argmax(dim=1) == label).sum().item()

    accuracy = correct / len(train_labels)
    scheduler_cls.step()
    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

# Evaluate
classification_head.eval()
correct = 0
with torch.no_grad():
    for img, label in DataLoader(test_dataset, batch_size=128, shuffle=False):
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        correct += (logits.argmax(dim=1) == label).sum().item()

test_accuracy = correct / len(test_labels)
print(f"Test Accuracy%: {test_accuracy*100:.4f}")

Epoch [1/100], Loss: 3.4937, Accuracy: 0.4237
Epoch [2/100], Loss: 3.0544, Accuracy: 0.5492
Epoch [3/100], Loss: 2.8101, Accuracy: 0.6053
Epoch [4/100], Loss: 2.5918, Accuracy: 0.6578
Epoch [5/100], Loss: 2.3303, Accuracy: 0.7158
Epoch [6/100], Loss: 2.0782, Accuracy: 0.7538
Epoch [7/100], Loss: 1.8342, Accuracy: 0.8002
Epoch [8/100], Loss: 1.5554, Accuracy: 0.8491
Epoch [9/100], Loss: 1.2786, Accuracy: 0.8847
Epoch [10/100], Loss: 1.0565, Accuracy: 0.9179
Epoch [11/100], Loss: 0.8176, Accuracy: 0.9469
Epoch [12/100], Loss: 0.6860, Accuracy: 0.9662
Epoch [13/100], Loss: 0.6101, Accuracy: 0.9734
Epoch [14/100], Loss: 0.5217, Accuracy: 0.9831
Epoch [15/100], Loss: 0.4395, Accuracy: 0.9867
Epoch [16/100], Loss: 0.3945, Accuracy: 0.9916
Epoch [17/100], Loss: 0.3326, Accuracy: 0.9940
Epoch [18/100], Loss: 0.2885, Accuracy: 0.9958
Epoch [19/100], Loss: 0.2317, Accuracy: 0.9982
Epoch [20/100], Loss: 0.1984, Accuracy: 0.9988
Epoch [21/100], Loss: 0.1882, Accuracy: 0.9976
Epoch [22/100], Loss: 